# text generation using fnet

In [11]:
!pip install datasets

In [12]:
!pip install torch[transformers]

In [13]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [14]:
from datasets import load_dataset

# Load the dataset splits explicitly
try:
    datasets = load_dataset("wikitext", "wikitext-2-raw-v1", split=['train', 'validation', 'test'])
    # Assign splits to variables for clarity
    train_dataset, validation_dataset, test_dataset = datasets
    
    # Print dataset info
    print("Dataset loaded successfully!")
    print(f"Number of training samples: {len(train_dataset)}")
    print(f"Number of validation samples: {len(validation_dataset)}")
    print(f"Number of test samples: {len(test_dataset)}")
    
    # Show an example
    print("\nSample from training data:\n", train_dataset[0])
    
except Exception as e:
    print("Error loading dataset:", e)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Error loading dataset: [{'expected': SplitInfo(name='test', num_bytes=1305088, num_examples=4358, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='test', num_bytes=5176698, num_examples=17432, shard_lengths=None, dataset_name='parquet')}, {'expected': SplitInfo(name='train', num_bytes=546500949, num_examples=1801350, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='train', num_bytes=1113622699, num_examples=3676136, shard_lengths=[1650675, 1661350, 364111], dataset_name='parquet')}, {'expected': SplitInfo(name='validation', num_bytes=1159288, num_examples=3760, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='validation', num_bytes=4607450, num_examples=15040, shard_lengths=None, dataset_name='parquet')}]


In [17]:
import re

def preprocess_text(sentence):
    text = sentence["text"].lower()
    text = re.sub("[^a-z?!.,]", " ", text)
    text = re.sub("\s\s+", " ", text)
    sentence["text"] = text.strip()
    return sentence

# Apply preprocessing
datasets["train"] = datasets["train"].map(preprocess_text)
datasets["test"] = datasets["test"].map(preprocess_text)
datasets["validation"] = datasets["validation"].map(preprocess_text)

# Filter out short texts
datasets["train"] = datasets["train"].filter(lambda x: len(x["text"]) > 20)
datasets["test"] = datasets["test"].filter(lambda x: len(x["text"]) > 20)
datasets["validation"] = datasets["validation"].filter(lambda x: len(x["text"]) > 20)


NameError: name 'datasets' is not defined

In [10]:

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding 
from transformers import AutoTokenizer 

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenizer
def tokenize(sentence):
    sentence = tokenizer(sentence['text'], truncation=True)
    return sentence

tokenized_inputs = datasets['test'].map(tokenize)
tokenized_inputs = tokenized_inputs.remove_columns(['text'])
# DataCollator
batch = 16
data_collator = DataCollatorWithPadding(
 
   tokenizer=tokenizer, padding=True, return_tensors="pt")

dataloader = DataLoader(tokenized_inputs, batch_size=batch, collate_fn=data_collator)



c:\Users\digit\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\digit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


NameError: name 'datasets' is not defined

In [18]:
# Step 5 : Adding the Positioning embedding

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft as fft
import numpy as np
import pandas as pd

class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.d_model = d_model
        self.max_sequence_length = max_sequence_length
        self.positional_encoding = self.create_positional_encoding().to(device)

    def create_positional_encoding(self):
        # Initialize positional encoding matrix
        positional_encoding = np.zeros((self.max_sequence_length, self.d_model))

        for pos in range(self.max_sequence_length):
           for i in range(0, self.d_model, 2):
        # For even indices apply sin
            positional_encoding=[pos, i] = np.sin(pos / (10000 ** ((2 * i) / self.d_model)))
        # For odd indices apply cos
            if i + 1 < self.d_model:
               positional_encoding=[pos, i + 1] = np.cos(pos / (10000 ** ((2 * i) / self.d_model)))

# Return positional encoding tensor
        return torch.from_numpy(positional_encoding).float()

def forward(self, x):
    # Add position embedding which is encoded in self.positional_encoding
    expanded_tensor = torch.unsqueeze(self.positional_encoding, 0).expand(x.size(0), -1, -1).to(device)

    return x.to(device) + expanded_tensor[:,:x.size(1), :]

class PositionalEmbedding(nn.Module):
  def __init__(self, sequence_length, vocab_size, embed_dim):
    super(PositionalEmbedding, self).__init__()
    self.token_embeddings = nn.Embedding(vocab_size, embed_dim)
    self.position_embeddings = PositionalEncoding(embed_dim,sequence_length)

  def forward(self, inputs):
    embedded_tokens = self.token_embeddings(inputs).to(device)
    embedded_positions = self.position_embeddings(embedded_tokens).to(device)
    return embedded_positions.to(device)



SyntaxError: cannot assign to expression (1998605791.py, line 27)

In [19]:
# Step 6 : Create FNet Encoder
class FNetEncoder(nn.Module):

  def __init__(self,embed_dim, dense_dim):
    super(FNetEncoder,self).__init__()
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.dense_proj = nn.Sequential(nn.Linear(self.embed_dim,self.dense_dim), nn.ReLU(), nn.Linear(self.dense_dim,self.embed_dim))

    self.layernorm_1 = nn.LayerNorm(self.embed_dim)
    self.layernorm_2 = nn.LayerNorm(self.embed_dim)

  def forward(self,inputs):

    fft_result = fft.fft2(inputs)

    #taking real part
    fft_real = fft_result.real.float()

    proj_input = self.layernorm_1 (inputs + fft_real)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input +proj_output)

NameError: name 'nn' is not defined

In [20]:
# Step 7 : Create FnetDecoder
class FNetDecoder(nn.Module):

  def __init__(self,embed_dim,dense_dim,num_heads):
    super(FNetDecoder,self).__init__()
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads

    self.attention_1 = nn.MultiheadAttention(embed_dim,num_heads,batch_first=True)
    self.attention_2 = nn.MultiheadAttention(embed_dim,num_heads,batch_first=True)

    self.dense_proj = nn.Sequential(nn.Linear(embed_dim, dense_dim),nn.ReLU(),nn.Linear(dense_dim, embed_dim))

    self.layernorm_1 = nn.LayerNorm(embed_dim)
    self.layernorm_2 = nn.LayerNorm(embed_dim)
    self.layernorm_3 = nn.LayerNorm(embed_dim)

  def forward(self, inputs, encoder_outputs, mask=None):
    causal_mask = nn.Transformer.generate_square_subsequent_mask(inputs.size(1)).to(device)

    attention_output_1, _ = self.attention_1(inputs, inputs, inputs, attn_mask=causal_mask)
    out_1 = self.layernorm_1(inputs + attention_output_1)

    if mask!= None:
      attention_output_2, _ = self.attention_2(out_1, encoder_outputs, encoder_outputs, key_padding_mask =torch.transpose(mask, 0, 1).to(device))
    else:
     attention_output_2, _ = self.attention_2(out_1, encoder_outputs, encoder_outputs)
    out_2 = self.layernorm_2(out_1 + attention_output_2)

    proj_output = self.dense_proj(out_2)
    return self.layernorm_3(out_2 + proj_output)

NameError: name 'nn' is not defined

In [21]:
# Step 8 : Fnet Model
class FNetModel(nn.Module):
    def __init__(self, max_length, vocab_size, embed_dim, latent_dim, num_heads):
        super(FNetModel, self).__init__()

        self.encoder_inputs = PositionalEmbedding(max_length,vocab_size, embed_dim)
        self.encoder1 = FNetEncoder(embed_dim, latent_dim)
        self.encoder2 = FNetEncoder(embed_dim, latent_dim)
        self.encoder3 = FNetEncoder(embed_dim, latent_dim)
        self.encoder4 = FNetEncoder(embed_dim, latent_dim)

        self.decoder_inputs = PositionalEmbedding(max_length,vocab_size, embed_dim)
        self.decoder1 = FNetDecoder(embed_dim, latent_dim, num_heads)
        self.decoder2 = FNetDecoder(embed_dim, latent_dim, num_heads)
        self.decoder3 = FNetDecoder(embed_dim, latent_dim, num_heads)
        self.decoder4 = FNetDecoder(embed_dim, latent_dim, num_heads)

        self.dropout = nn.Dropout(0.5)
        self.dense = nn.Linear(embed_dim, vocab_size)

    def encoder(self,encoder_inputs):
        x_encoder = self.encoder_inputs(encoder_inputs)
        x_encoder = self.encoder1(x_encoder)
        x_encoder = self.encoder2(x_encoder)
        x_encoder = self.encoder3(x_encoder)
        x_encoder = self.encoder4(x_encoder)
        return x_encoder

    def decoder(self,decoder_inputs,encoder_output,att_mask):
        x_decoder = self.decoder_inputs(decoder_inputs)
        x_decoder = self.decoder1(x_decoder, encoder_output,att_mask) ## HERE for inference
        x_decoder = self.decoder2(x_decoder, encoder_output,att_mask) ## HERE for inference
        x_decoder = self.decoder3(x_decoder, encoder_output,att_mask) ## HERE for inference
        x_decoder = self.decoder4(x_decoder, encoder_output,att_mask) ## HERE for inference
        decoder_outputs = self.dense(x_decoder)

        return decoder_outputs

    def forward(self, encoder_inputs, decoder_inputs,att_mask = None):
        encoder_output = self.encoder(encoder_inputs)
        decoder_output = self.decoder(decoder_inputs,encoder_output,att_mask=None)
        return decoder_output

NameError: name 'nn' is not defined

In [22]:
# Step 9 : Initialize Model
# Assuming your constants are defined as follows:
MAX_LENGTH = 512
VOCAB_SIZE = len(tokenizer.vocab)
EMBED_DIM = 256
LATENT_DIM = 100
NUM_HEADS = 4

# Instantiate the model
fnet_model = FNetModel(MAX_LENGTH, VOCAB_SIZE, EMBED_DIM, LATENT_DIM, NUM_HEADS).to(device)

NameError: name 'FNetModel' is not defined

In [23]:
# Step 10 : Train the model
# Define your optimizer and loss function
optimizer = torch.optim.Adam(fnet_model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

epochs = 100
for epoch in range(epochs):
    train_loss = 0
    for batch in dataloader:
        encoder_inputs_tensor = batch['input_ids'][:,:-1].to(device)
        decoder_inputs_tensor = batch['input_ids'][:,1:].to(device)

        att_mask = batch['attention_mask'][:,:-1].to(device).to(dtype=bool)
        optimizer.zero_grad()
        outputs = fnet_model(encoder_inputs_tensor, decoder_inputs_tensor,att_mask)
        decoder_inputs_tensor.masked_fill(batch['attention_mask'][:,1:].ne(1).to(device), -100).to(device)

        loss = criterion(outputs.view(-1, VOCAB_SIZE), decoder_inputs_tensor.reshape(-1))
        train_loss = train_loss + loss.item()
        loss.backward()
        optimizer.step()
print (f"epoch: {epoch}, train_loss : {train_loss}")


NameError: name 'fnet_model' is not defined